In [1]:
!pip install evaluate rouge_score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8a36feb1661982ffcf407a8671abdaf39dbfde2b92ac927b96b3c109b2a7b322
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import evaluate

In [3]:
# 1. دیتاست (wmt14 → انگلیسی-فرانسوی)
dataset = load_dataset("wmt14", "fr-en")

# 2. مدل و توکنایزر (مدل مخصوص en→fr)
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

fr-en/train-00000-of-00030.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

fr-en/train-00001-of-00030.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [4]:
train_dataset = dataset["train"].select(range(1000))
val_dataset = dataset['validation'].select(range(250))
val_dataset

Dataset({
    features: ['translation'],
    num_rows: 250
})

In [5]:
# 3. پیش‌پردازش داده‌ها
max_input_length = 128
max_target_length = 128

def preprocess(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
tokenized_train_dataset = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val_dataset = val_dataset.map(
    preprocess,
    batched=True,
    remove_columns=val_dataset.column_names
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [7]:
# 4. Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
# 5. متریک BLEU
bleu = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = bleu.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    return {"bleu": result["score"]}

In [9]:
# 6. تنظیمات آموزش
training_args = Seq2SeqTrainingArguments(
    output_dir="./translation-model",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none"
)

In [10]:
# 7. Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # نمونه کوچک برای تست
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-852756518.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
# 8. شروع آموزش
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,3.059700,2.525351,39.302685


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=125, training_loss=2.951935211181641, metrics={'train_runtime': 61.9695, 'train_samples_per_second': 16.137, 'train_steps_per_second': 2.017, 'total_flos': 17716132970496.0, 'train_loss': 2.951935211181641, 'epoch': 1.0})

In [14]:
# 9. تست روی یک جمله
sentence = "I love machine learning and natural language processing."
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=128)

# Move input tensors to the same device as the model
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

translated_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4)
print("English:", sentence)
print("French:", tokenizer.decode(translated_ids[0], skip_special_tokens=True))

English: I love machine learning and natural language processing.
French: J'adore l'apprentissage de la machine et le traitement du langage naturel.
